In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


### **Load the train model, scaler pickle, onehot**

In [21]:
# Load the train model
model = load_model('model.h5')

# Load the scaler pickle, onehot
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('OneHot_encoder_geo.pkl','rb') as file:
    OneHot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


### **Example input data**

In [22]:
# Example input data
input_data = {
    'creditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    "EstimatedSalary":50000
}

### **One-Hot encoder 'Geograaphy'**

In [23]:
geo_encoder = OneHot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder,columns=OneHot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

f:\Deep Learning\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [24]:
input_data = pd.DataFrame([input_data])
input_data

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


<!-- Combine one-hot encoder columns with input data -->

### **encoded Categorical Variable**

In [26]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,creditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


### **Concatination one hot encoded**

In [27]:
input_data = pd.concat([input_data.drop("Geography",axis=1),geo_encoder_df],axis=1)
input_data

,creditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


### **Scaling the data**

In [35]:
input_data.rename(columns={'creditScore': 'CreditScore'}, inplace=True)
input_scaled = scaler.transform(input_data)

In [36]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

### **Predict Churn**

In [37]:
prediction =model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


In [38]:
prediction

array([[0.01917767]], dtype=float32)

In [39]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.019177673)

In [41]:
if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    {
    print("The customer is not likely to churn.")
}

The customer is not likely to churn.
